In [5]:
import gurobipy as gurobi
from gurobipy import GRB
from collections import OrderedDict
def build_name(pair):
    if isinstance(pair, str):
        return pair
    assert len(pair) == 2 and isinstance(pair, tuple)
    return str(pair)

def createPlayerBase(p):
    result = []
    for i in p:
        if i['type'] == 'decision':
            for j in i["actions"]:
                var = ((i['id'], j))
                # we directly add a positive value here...
                result.append(var)
    return result

def createPrototypeVar(m, p, constraint_type):
    result = OrderedDict()
    for i in p:
        if i['type'] == 'decision':
            for j in i["actions"]:
                var = ((i['id'], j))
                # we directly add a positive value here...
                if constraint_type == "binary":
                    result[var] = m.addVar(0.0, 1.0, vtype=GRB.BINARY, name=build_name(var))
                else:
                    assert constraint_type == "continuous"
                    result[var] = m.addVar(0.0, GRB.INFINITY, vtype=GRB.CONTINUOUS, name=build_name(var))
    return result

def wrap_dict(u, p1, p2):
    def helper(v1, v2):
        if (v1, v2) in u:
            return u[(v1,v2)]
        return 0
    return helper, p1, p2

def createPrototypeMatrix(game, p1, p2):
    u = game['utility_pl1']
    u = {(i["sequence_pl1"], i["sequence_pl2"]) : i['value'] for i in u}
    return wrap_dict(u, p1, p2)

def create_f(m, p):
    result = OrderedDict()
    for i in p:
        if i['type'] == 'decision':
            for j in i["actions"]:
                var = ((i['id'], j))
                # we directly add a positive value here...
                result[var] = m.addVar(0.0, GRB.INFINITY, vtype=GRB.CONTINUOUS, name=build_name(var))
    return result
    
def transpose(M):
    MM, p1, p2 = M
    return lambda x, y : MM(y, x), p2, p1

def negM(M):
    MM, p1, p2 = M
    return lambda x, y : - MM(x, y), p1, p2


def vector_mul(M, x):
    MM, p1, p2 = M
    result = OrderedDict()
    assert list(x.keys()) == p2
    for j in p1:
        cur = 0
        for i in x:
            cur += MM(j, i) * x[i]
        result[j] = cur
    return result

def vector_sub(v1, v2):
    assert list(v1.keys()) == list(v2.keys())
    x = OrderedDict()
    for i in v1:
        x[i] = v1[i] - v2[i]
    return x


def vector_dot(v1, v2):
    assert list(v1.keys()) == list(v2.keys())
    result = 0
    for i in v1:
        result += v1[i] * v2[i]
    return result

def createFf(p, p_base):
    f = OrderedDict()
    F = OrderedDict()
    iss = []
    for i in p:
        if i['type'] == 'decision':
            iss.append(i['id'])
            # collect children
            for j in i["actions"]:
                F[(i['id'], (i['id'], j))] = 1
            if i['parent_sequence'] is None:
                f[i['id']] = 1
            else:
                # a summation
                f[i['id']] = 0
                F[(i['id'], i['parent_sequence'])] = -1
    return wrap_dict(F, iss, p_base), f

def create_free_v(m, v):
    result = OrderedDict()
    for i in v:
        result[i] = m.addVar(-GRB.INFINITY, GRB.INFINITY, vtype=GRB.CONTINUOUS, name=build_name(i))
    return result

def addEqConstraint(m, lhs):
    for i in lhs:
        m.addConstr(lhs[i] == 0)
def addGTConstraint(m, lhs):
    for i in lhs:
        m.addConstr(lhs[i] >= 0)
def solve(game, constraint_type):
    # we represent a symbolic vector as a mapping from action point -> symbolic value
    p1 = game['decision_problem_pl1']
    p2 = game['decision_problem_pl2']

    p1_base = createPlayerBase(p1)
    p2_base = createPlayerBase(p2)

    A = createPrototypeMatrix(game, p1_base, p2_base)
    F1, f1 = createFf(p1, p1_base)
    F2, f2 = createFf(p2, p2_base) 
    # solve for player 1
    def solve_p1():
        m = gurobi.Model("game_value_pl1")
        x1 = createPrototypeVar(m, p1, constraint_type=constraint_type)
        free_v2 = create_free_v(m, f2)
        LHS1 = vector_sub(vector_mul(transpose(A), x1), vector_mul(transpose(F2), free_v2))
        LHS2 = vector_sub(vector_mul(F1, x1), f1)
        addGTConstraint(m, LHS1)
        addEqConstraint(m, LHS2)
        target = vector_dot(free_v2, f2)
        m.setObjective(target, sense=GRB.MAXIMIZE)
        m.optimize()
        return m.ObjVal
    def solve_p2():
        m = gurobi.Model("game_value_pl2")
        x2 = createPrototypeVar(m, p2, constraint_type=constraint_type)
        free_v1 = create_free_v(m, f1)
        LHS1 = vector_sub(vector_mul(negM(A), x2), vector_mul(transpose(F1), free_v1))
        LHS2 = vector_sub(vector_mul(F2, x2), f2)
        addGTConstraint(m, LHS1)
        addEqConstraint(m, LHS2)
        target = vector_dot(free_v1, f1)
        m.setObjective(target, sense=GRB.MAXIMIZE)
        m.optimize()
        return m.ObjVal
    print(solve_p1())
    print(solve_p2())
def helper(path, constraint_type):
    import json
    game = json.load(open(path))

    # Convert all sequences from lists to tuples
    for tfsdp in [game["decision_problem_pl1"], game["decision_problem_pl2"]]:
        for node in tfsdp:
            if isinstance(node["parent_edge"], list):
                node["parent_edge"] = tuple(node["parent_edge"])
            if "parent_sequence" in node and isinstance(node["parent_sequence"], list):
                node["parent_sequence"] = tuple(node["parent_sequence"])
    for entry in game["utility_pl1"]:
        assert isinstance(entry["sequence_pl1"], list)
        assert isinstance(entry["sequence_pl2"], list)
        entry["sequence_pl1"] = tuple(entry["sequence_pl1"])
        entry["sequence_pl2"] = tuple(entry["sequence_pl2"])
    solve(game, constraint_type=constraint_type)
for constraint_type in ["continuous", "binary"]:
    for game in ["lp_files/rock_paper_superscissors.json", "lp_files/kuhn_poker.json", "lp_files/leduc_poker.json"]:
        helper(game, constraint_type=constraint_type)


Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (linux64 - "Manjaro Linux")

CPU model: 12th Gen Intel(R) Core(TM) i3-12100F, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 4 rows, 4 columns and 12 nonzeros
Model fingerprint: 0x129a13d1
Coefficient statistics:
  Matrix range     [1e+00, 2e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 1e+00]
Presolve removed 1 rows and 0 columns
Presolve time: 0.00s
Presolved: 3 rows, 4 columns, 11 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.0000000e+00   1.000000e+00   0.000000e+00      0s
       2   -0.0000000e+00   0.000000e+00   0.000000e+00      0s

Solved in 2 iterations and 0.00 seconds (0.00 work units)
Optimal objective -0.000000000e+00
-0.0
Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (linux64 - "Manjaro Linux")

CPU model: 12th Gen Intel(R) Core(TM) i3-12100F

In [5]:
m.ObjVal

-0.055555555555555525

In [35]:
Ff1 = createFf(p1, p1_base)

[{'sequence_pl1': ('d1_pl1', 'r'),
  'sequence_pl2': ('d1_pl2', 'p'),
  'value': -1},
 {'sequence_pl1': ('d1_pl1', 'r'),
  'sequence_pl2': ('d1_pl2', 's'),
  'value': 1},
 {'sequence_pl1': ('d1_pl1', 'p'),
  'sequence_pl2': ('d1_pl2', 'r'),
  'value': 1},
 {'sequence_pl1': ('d1_pl1', 'p'),
  'sequence_pl2': ('d1_pl2', 's'),
  'value': -2},
 {'sequence_pl1': ('d1_pl1', 's'),
  'sequence_pl2': ('d1_pl2', 'r'),
  'value': -1},
 {'sequence_pl1': ('d1_pl1', 's'),
  'sequence_pl2': ('d1_pl2', 'p'),
  'value': 2}]